In [2]:
import configs as cfgs
import math_util as mt
import data_util as dt
import logging
import numpy as np

from trainer import set_custom_object
from keras.models import load_model
from keras.optimizers import RMSprop, Adam, SGD
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

Using TensorFlow backend.


In [3]:
# path of test
path_list = cfgs.base_data_path + "val_list.txt"
###
f = open(path_list)
path_list = f.readlines()
f.close()

path_imgs = []
path_rois = []
for i in range(0, len(path_list)-1, 2):
    if path_list[i][-1:] is "\n":
        path_list[i] = path_list[i][:-1]
        path_list[i+1] = path_list[i+1][:-1]
    
    path_imgs.append(path_list[i])
    path_rois.append(path_list[i+1])

for i in range(len(path_imgs)):
    print("[{:}] {:}".format(i, path_imgs[i]))

[0] ./dataset/raw_images\20349_3_11_B31_norm_hu.npz
[1] ./dataset/raw_images\20349_3_12_B31_norm_hu.npz
[2] ./dataset/raw_images\20349_3_13_B31_norm_hu.npz
[3] ./dataset/raw_images\20349_3_14_B31_norm_hu.npz
[4] ./dataset/raw_images\20349_3_15_B31_norm_hu.npz
[5] ./dataset/raw_images\20349_3_16_B31_norm_hu.npz


In [4]:
print("See number of your file name on above")
user_pick = input() # if more than 1 use form 1, 2, 3, 4  ** -1 for all

See number of your file name on above
-1


In [6]:
test_pick = user_pick.split(", ")
test_pick = list(map(int, test_pick))

if -1 in test_pick:
    test_pick = [num for num in range(0, len(path_imgs))]

logging.info("Your pick is.. {:}".format(test_pick))

2018-11-26 14:21:58,994 Your pick is.. [0, 1, 2, 3, 4, 5]


In [8]:
# select
batch_size = cfgs.predict_batch_size
mt.threshold = 0.7

In [9]:
best_model = dt.all_path(cfgs.best_model_path+"*")
num_epoch, best_model = dt.find_best_path(best_model)

logging.info("Load model..")
loss = mt.get_loss()
metrics = mt.get_metrics()
set_custom_object(loss, metrics)
model = load_model(best_model)

logging.info("epoch: {:} path: {:}".format(num_epoch, best_model))

2018-11-26 14:22:09,139 Load model..
2018-11-26 14:24:19,792 epoch: 18 path: ./t_model/best\epoch_018-hard_dice_coef_th_0.9315.hdf5


In [10]:
optimizer = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0, amsgrad=False)
#optimizer = SGD(lr=1e-2, momentum=0.9, decay=0, nesterov=True)
#optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(optimizer=optimizer, 
              loss=loss,
              metrics=[mt.get_function("hard_dice_coef_th"), 
                       mt.get_function("precision_th"), 
                       mt.get_function("recall_th"), 
                       mt.get_function("f1_th")])

In [ ]:
logging.info("Start")
all_sc = []



In [11]:
logging.info("Start")
all_sc = []

for i in(test_pick):
    op_imgs = dt.process_data(dt.open_im_roi(path_imgs[i]) )
    op_rois = dt.open_im_roi(path_rois[i])
    
    for j in range(op_imgs.shape[2]):
        img = dt.divide_image(op_imgs[...,j])
        roi = dt.process_roi(dt.divide_image(op_rois[...,j]) )
        
    h = model.evaluate(x=eval_imgs, y=eval_rois, batch_size=batch_size, verbose=1)
    
    for j in(h):
        all_sc.append(j)
    
    name = dt.tail_word(path_imgs[i])
    name = name[:name.find(".")]
    logging.info("Complete.. " + name)
    
logging.info("All Completed!")

2018-11-26 14:24:30,139 Start


814/814 [==============================] - 126s 155ms/step


2018-11-26 14:28:06,215 Complete.. 20349_3_11_B31_norm_hu


699/699 [==============================] - 70s 101ms/step


2018-11-26 14:30:17,403 Complete.. 20349_3_12_B31_norm_hu


...

2018-11-26 14:35:15,746 Complete.. 20349_3_14_B31_norm_hu


757/757 [==============================] - 53s 70ms/step


2018-11-26 14:37:06,059 Complete.. 20349_3_15_B31_norm_hu


671/671 [==============================] - 56s 83ms/step


2018-11-26 14:39:27,261 Complete.. 20349_3_16_B31_norm_hu
2018-11-26 14:39:31,581 All Completed!


In [20]:
print("==========Scores of each==========")
all_total = []
for i in range(len(all_sc)):
    print(model.metrics_names[i%len(model.metrics_names)] + ": {:.4f}".format(all_sc[i]) )
    if i < len(model.metrics_names):
        all_total.append(all_sc[i])
    else:
        all_total[i%len(model.metrics_names)] += all_sc[i]
    if i!=0 and i%len(model.metrics_names)==0:
        print("==================================")
print("==========Total scores of all==========")        
for i in range(len(all_total)):
    print(model.metrics_names[i%len(model.metrics_names)] + ": {:.4f}".format(all_total[i]/(len(all_sc)/len(model.metrics_names))) )

==========Scores of each==========
loss: 0.0064
hard_dice_coef_th: 0.8769
precision_th: 0.6086
recall_th: 0.6438
f1_th: 0.6207
loss: 0.0122
hard_dice_coef_th: 0.9108
precision_th: 0.8228
recall_th: 0.8590
f1_th: 0.8372
loss: 0.0035
hard_dice_coef_th: 0.9320
precision_th: 0.8956
recall_th: 0.8639
f1_th: 0.8795
loss: 0.0021
hard_dice_coef_th: 0.9023
precision_th: 0.9098
recall_th: 0.8875
f1_th: 0.8985
loss: 0.0024
hard_dice_coef_th: 0.9303
precision_th: 0.9203
recall_th: 0.9071
f1_th: 0.9137
loss: 0.0096
hard_dice_coef_th: 0.9674
precision_th: 0.9262
recall_th: 0.9105
f1_th: 0.9183
==========Total scores of all==========
loss: 0.0060
hard_dice_coef_th: 0.9199
precision_th: 0.8472
recall_th: 0.8453
f1_th: 0.8446
